In [ ]:
import jax.random as jrandom
import numpy as np
import os

from tqdm.notebook import tqdm

from jaxl.constants import *
from jaxl.models.utils import get_model, load_config, load_params, get_wsrl_model, iterate_params, get_policy, policy_output_dim
from jaxl.envs import get_environment
from jaxl.envs.rollouts import EvaluationRollout
from jaxl.utils import get_device, parse_dict

# get_device("gpu:0")
get_device("cpu")

In [ ]:
result_dir = "/home/bryan/research/jaxl/logs/manipulator_learning"

ablation_name = "stack"
learner_name = "cross_q-sac-06-03-24_16_53_41-9dd96b95-aefd-44fd-8894-4854c9c08abf"
learner_name = "bc-06-04-24_09_43_02-a09b01c4-e33d-4e88-9eda-d7d36a68cdb8"
# learner_name = "bc-100k_steps-06-04-24_09_57_35-e8bd5a54-9148-41a9-ace8-f33c5cfbab9f"
# learner_name = "bc-10k_steps-06-04-24_10_06_35-333b32a8-c019-4fed-9b8f-1ce59166bb2b"
# learner_name = "warm_start_reinforce-06-04-24_13_28_32-b356a022-d53a-4b11-9726-ccfe4dca0777"
learner_name = "rlpd-sac-06-05-24_16_15_56-c5ad96da-4ac4-466b-a221-74cfea71bd19"

learner_path = os.path.join(result_dir, ablation_name, learner_name)

In [ ]:
_, config = load_config(learner_path)
config

In [ ]:
env_config = {
    "env_type": "manipulator_learning",
    "env_name": "PandaPlayInsertTrayXYZState",
    "env_kwargs": {
        "main_task": "stack",
        "dense_reward": False
    }
}
env = get_environment(parse_dict(env_config))

In [ ]:
model_out_dim = policy_output_dim(env.act_dim, config.learner_config)

if config.learner_config.learner == CONST_BC:
    model = get_model(
        int(np.prod(env.observation_space.shape)) + 1,
        env.act_dim,
        config.model_config
    )
    include_absorbing_state = True
elif config.learner_config.task == CONST_WSRL:
    model = get_wsrl_model(
        env.observation_space.shape,
        model_out_dim,
        config.model_config.policy
    )
    include_absorbing_state = True
else:
    model = get_model(
        env.observation_space.shape,
        model_out_dim,
        config.model_config.policy
    )
    include_absorbing_state = False
params = load_params(f"{learner_path}:latest")

policy = get_policy(model, config.learner_config)

In [ ]:
total_episodes = 50
eval_seed = 42
render = True
random = True

In [ ]:
rollout = EvaluationRollout(env, eval_seed)

In [8]:
rollout.rollout(
    params[CONST_MODEL_DICT][CONST_MODEL][CONST_POLICY],
    policy,
    False,
    total_episodes,
    random=random,
    render=render,
    include_absorbing_state=include_absorbing_state,
)

In [ ]:
np.mean(rollout.episodic_returns), np.std(rollout.episodic_returns)

10k: (203.98, 147.88853775732588)  
100k: (253.7, 127.34115595517422)  
1M: (215.96, 148.52123888521803)